### 마스킹 대상 파일 및 컬럼생성

In [ ]:
#마스킹 대상 파일 소스 위치 to list
import os
allList=[]
fileList=[]
f=open('c:/kpayins/temp/마스킹대상.txt',encoding='utf8')
findl=[x.replace('\n','') for x in f.readlines()]

for root,dirs,files in os.walk('C:\kis\workspace\kis-ccr'):
#     print('dirs=',root,dirs)
    for fname in files:
        full_name=os.path.join(root,fname)
        if fname in findl:  #대상파일
            fileList.append(full_name)
        allList.append(full_name) #모든 파일

In [ ]:
#각 파일별 io name to list
ioName=[]
dic={}

for file in fileList:
    f=open(file,'r',encoding='utf-8')
    while True:
        ln=f.readline()
        if not ln: break
        if ('public' in ln) & ('doProcessInternal(' in ln):  #소스상 io선언
            idx=ln.find('doProcessInternal(')+18
            ky=file.split('\\')[-1:][0]
            dic[ky]=ln[idx:idx+9]+'.java'   #io파일명 추출


In [ ]:
# iofile_Dict 생성
ioLis=dic.values()
iofile_List=[a.split('\\')[-1:][0] for i in ioLis for a in allList if i in a]
iofile_Dict=dict([(k,a) for k,i in dic.items() for a in allList if i in a])
# iofile_Path=[a for i in ioLis for a in allList if i in a]
print('notfnd=',[x for x in ioLis if x not in iofile_List])

In [ ]:
# iofile_Dict 읽어 마스킹 대상 필드를 식별한다

In [ ]:
tbl='마스킹컬럼.xlsx'
path='c:/kpayins/11.Tobe분석/'
read_file=path+tbl
df_mask=pd.read_excel(read_file,sheet_name='컬럼',header=0)

In [ ]:
import re
def evaluate_field(fld_nm,df):
    for i,row in df.iterrows():
        if row['pattern']=='단어일치':
            p=re.compile(f"^{row['column_nm']}$",re.IGNORECASE)
        elif row['pattern']=='끝단어':   
            p=re.compile(f"\w*{row['column_nm']}$",re.IGNORECASE)
        elif row['pattern']=='단어포함':   
            p=re.compile(f"\w*{row['column_nm']}\w*",re.IGNORECASE)
        else: 
            print('error',row)
            return None
    
        if re.match(p,fld_nm):
#             print('succ',row,fld_nm)
            return row['type']
    return None


In [ ]:
##vo를 읽어 필드명,  id, 길이를 추출  
import os,re
import pandas as pd
# dd={'SvCcrD020.java': 'C:\\kis\\workspace\\kis-ccr\\kis-ccr-shared\\kis-ccr-Cr\\src\\main\\java\\com\\kakaopay\\ccr\\Cr\\Cmn\\vo\\IoCcrD020.java'}
df_ListAll=pd.DataFrame()

for file_nm,path in iofile_Dict.items():
    dic={}
    f=open(path,'r',encoding='utf-8')
     #대상파일 read 및 처리    
    lines=f.readlines()
    startp=0
#     print(path,lines)
    io_nm=path.split('\\')[-1:][0]
    for i,line in enumerate(lines):
        if line.find('public class')>=0:
            startp=1 
        elif line.find('public')>=0:
             break

#         print(line)
        if startp==1:
            line=line.strip()
            ss= line[line.find('(')+1:line.find(')')].replace('"','')
            if line.find('@JsonProperty')>=0:  #컬럼정보
                dic['file']=file_nm
                dic['io']=io_nm
                dic['line']=i+1
                dic['c_id']=ss
            if line.find('@LocalName')>=0:    #컬럼한글
                dic['c_nm']=ss
            if line.find('private')>=0:     #컬럼정보 끝나면 해당컬럼이 마스킹 대상인지 판별
                res=evaluate_field(dic['c_id'],df_mask)
                if res:
                    dic['type']=res
                    df_ListAll=df_ListAll.append(dic,ignore_index=True)
                dic={}
                
df_ListAll=df_ListAll.append(dic,ignore_index=True)
            
# print(df_ListAll)   
wrt_file='c:/kpayins/temp/마스킹대상서비스io.xlsx'
with pd.ExcelWriter(os.path.join(wrt_file)) as writer:  
         df_ListAll.to_excel(writer, sheet_name='마스킹대상',index=False),
        

f.close()
print('job complted..')

### 암복호화 미적용->적용

In [ ]:
tbl='암호화컬럼_CRPTO_ERR.xlsx'
path='c:/kpayins/temp/'
read_file=path+tbl
df_enc=pd.read_excel(read_file,sheet_name='data',header=0)
df_enc.fillna(method='ffill',inplace=True)
df_enc['file_nm']=df_enc['Class'].apply(lambda x: x.split('.')[-1:][0])+'.java'
exp='Field=="@class" & 대상==True & 실제==False'  #미적용된 vo class
df_mod = df_enc.query(exp)
voLis=list(df_mod['file_nm'].unique())
# voLis=list(df_enc['file_nm'].unique())

In [ ]:
# 전체 파일리스트에서 해당 class 파일 찾아서 파일명,경로 dict생성
vofile_List=[(a.split('\\')[-1:][0],a) for i in voLis for a in allList if i in a]
dic_vo=dict(vofile_List)

In [ ]:
# 미적용 class파일을 열어 적용하여 출력
for i,row in df_mod.iterrows():
# for i,row in df_enc2.iterrows():  #암호화 누락class
    file=dic_vo.get(row['file_nm']) #file open 및 출력파일
    f=open(file,'r',encoding='utf-8')
    lines=f.readlines()
    fw=open(f"c:/kpayins/temp/{row['file_nm']}",'w',encoding='utf-8')
    exp=f"Class=='{row['Class']}' & Field !='@class'"
    df_t=df_enc.query(exp)  #누락 컬럼정보
    
    if len(df_t[df_t.실제==True])==0:
        import_yn =True  # 모두 미적용이면 crypto import
    else: 
        import_yn =False

    for i,line in enumerate(lines):
        if ('import' in line) & (import_yn==True):
            fw.write(f'import com.kakaopay.framework.crypto.Crypto;\n') #1.import crypto적용
            import_yn =False
        if line.find('public class')>=0:                                #2.class crypto적용
            fw.write(f'@Crypto\n')      
            startp=True
        elif line.find('public')>=0:
            startp=False

        if startp==True:
            for i,row in df_t.iterrows():
                if row['Field']=="@class":
                    continue
                if (row['Field'] in line) & (row['대상']==True) & (row['실제']==False):
                    fw.write(f'\t@Crypto\n')                        #3. 컬럼 crypto적용
        fw.write(line)

    f.close()
    fw.close()
    
print('completed')

### 특정 KEYWORD가 들어있는 라인을 추출

In [ ]:
#각 파일을 읽어 특정 KEYWORD가 들어있는 라인을 추출
keyline=[]
dic={}
keywd='umsUtil.send' #ums발송하는 업무

for file in allList:
    if '.java' not in file:
         continue

    f=open(file,'r',encoding='utf-8')
    lineNum=0
    ky=str(file.split('\\')[-1:])
    ky=re.sub('[\[|\]|\']','',ky)  # [ ] ' 제거
    while True:
        ln=f.readline()
        if not ln: break
        lineNum += 1
        if keywd in ln:  #keyword가 포함된 파일
            ln=ln.strip()
            keyline.append(f'{ky}:{str(lineNum)}:{ln}')


In [ ]:
df=pd.DataFrame([i.split(':') for i in keyline])
df.to_excel('c:/kpayins/temp/ums발송목록.xlsx')